In [1]:
import pandas as pd
# import os
# import lxml
# import DB as db

# Test - 1 

In [ ]:
projects= (66, 67, 68, 69)
# files = ('Results-66.txt', 'Results-67.txt', 'Results-68.txt', 'Results-69.txt')
dir = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\change land-no\checksum"

lst = []
for p in projects:

    fpath = os.path.join(dir, f"Results-{p}.txt") 
    with open(fpath, 'rt') as f:
        while(True):
            line = f.readline()
            if not line: break
            if line[:5] in ('-----', '     '): continue
            NID = line[25:39]
            land_id = line[51:56]
            res_no = line[79:93]
            valid = line[98:106].strip()
            rec = [p, NID, land_id, res_no, valid]
            lst.append(rec)

df = pd.DataFrame(data=lst, columns=['project', 'NID', 'land_id', 'reservation_no', 'valid'])
df.to_csv('./out/checksum.csv')

In [ ]:
df1=pd.DataFrame()
df1['NID'] = df.iloc[:,0].str[25:41]
df1['land_id'] = df.iloc[:,1].str[10:16]
df1['res_no'] = df.iloc[:,2].str[22:35]
df1['valid'] = df.iloc[:,2].str[40:50]
df1.loc[df1.NID.str.startswith('---')]
# df1

In [ ]:
dblog_csv_fn = r"C:\Yahia\Python\portal_logs\out\dblog_from-to_db_log.csv"
# df = pd.read_csv(dblog_csv_fn, parse_dates=True, usecols=['tmstamp', 'query','params','proc_tbl'])
df = pd.read_csv(dblog_csv_fn, parse_dates=True, usecols=['params','proc_tbl'])
proc_name = 'reserveLand'
tbl = df.loc[df.proc_tbl == proc_name]    

In [ ]:
t = tbl.params.str.split(',')
# t.apply(lambda x: x[2])
t.apply(lambda x: len(x))


In [ ]:
nparams = len(tbl.iloc[0,1].split(','))
df = pd.DataFrame()
t = tbl.params.str.split(',')
for i in range(nparams) :
    df[f"P{i+1}"] = t.apply(lambda x: x[i])

df

In [ ]:
{"Gov":"القاهرة","City":"القاهرة الجديدة","Region":"نزهة التجمع الثالث بمنطقة المستقبل (3)","District":"null","Sub_District":"null","Floor_No":"0","building_no":"2","Unit_No":"15","Unit_Model":"A","Unit_ID":"118744","checksum":"01AE4AA44C9C78B81EFC0443D02AC43C687F023ECCB8A07631FEC86258B50CE7F6"}

In [ ]:
import tarfile
fn = r"C:\Users\yahia\Downloads\server.log.2022-12-28.tar.gz"

file_name = None
file_obj = fn
with tarfile.open(name  = fn, fileobj = file_obj, mode = "r:gz") as tar:
    for member in tar.getmembers():
        f=tar.extractfile(member)
        for f in zfiles:
        line_no = 0
        # print (fname)
        while True:
            txt = f.readline()
            # print (len(txt), txt)
            if not txt: break # end of file
            line_no += 1

            # if line_no > 10000: break
            # if line_no < 100000000: continue
            # if line_no % 10000 == 0: print (line_no)
            if line_no % 40000 == 0: print (line_no)
            
            txt = txt.decode('utf-8')
            log_type = txt[24:29]

In [ ]:

import pandas as pd
fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\log summary-2022-12-26.zip"
df = pd.read_csv(fn, compression='zip', date_parser='timestamp', infer_datetime_format=True)

In [ ]:
# x= pd.to_datetime(df.timestamp).dt.date
# df1 = pd.concat([df,x], axis=1)
df1=df.copy()
df1.timestamp = pd.to_datetime(df.timestamp).dt.date


In [ ]:
df1.groupby('timestamp').count().max().index

# Test -2

In [ ]:

fn = r"./out/xx.csv"
df = pd.read_csv(fn)    #, compression='zip')#, date_parser='timestamp', infer_datetime_format=True)

In [ ]:
# df['res_date'] = df.land_application_date.apply(lambda x: x[:10])
df1= df[['project_id', 'land_application_date']].groupby(['project_id']).count()
df1 = df1.rename({'land_application_date':'Count'}, axis=1).reset_index().set_index('project_id')
df2 = db.query_to_pd('project')
df3 = pd.concat([df2, df1],axis=1 )
df3.dropna(subset=['Count'], inplace=True)
df3


In [ ]:
df3

# Test 3    

## Type 1

In [ ]:

# fn = r"C:\Users\yahia\Downloads\18-06.txt"
fn = r"C:\Users\yahia\Downloads\server.log.2023-01-22"
out_lst = []
with  open (fn, 'rt', encoding='utf8') as f:
    line_no = 1

    while True:
        txt = f.readline()
        if not txt:
            break

        # if txt.find('WebRequestInterceptor') != -1: continue
        # x = txt.find("[eg.intercom.hdb.rer.web.controllers.advice.ControllerExceptionHandler]")
        # if x == -1:
            # line_no +=1
            # continue
        if txt[30:101] != "[eg.intercom.hdb.rer.web.controllers.advice.ControllerExceptionHandler]":
            # print (txt[101:171])
            continue
        
        lst = txt[122:].rstrip().split(': ')
        l = len(lst)
        try:
            if l ==1:
                exception_type = lst[0].lstrip()
                exception_desc = ''
            elif l == 2:
                exception_type = lst[1]
                exception_desc = ''
            else:
                exception_type = lst[1]
                exception_desc = lst[2]
        except:
            print (line_no, ":", len(lst) , " ************\n", txt )
        x = exception_desc.find('org.springframework')
        if x != -1: # found
            y = exception_desc[x:].find(':')
            exception_desc_type = exception_desc[x:y].split('.')[-1]
        else:
            exception_desc_type = ''
        out_lst.append([line_no, exception_type, exception_desc_type, exception_desc])
        line_no +=1
        # if line_no > 20000: break
        if line_no % 1000 == 0: print (line_no)
df = pd.DataFrame(out_lst, columns=['line_no', 'exception_type', 'exception_desc_type', 'exception_desc'])
# x =df.pivot_table(columns=  'excption_type', index='excption_desc', values = 'line_no', aggfunc='count', margins=True, fill_value=0)
x =df.pivot_table(index= ['exception_type',  'exception_desc_type', 'exception_desc'], values = 'line_no', aggfunc='count', margins=True, fill_value=0)
x.to_csv('./out/xx1.csv', index=True)
# x


## Type-2

In [ ]:

# fn = r"C:\Users\yahia\Downloads\18-06.txt"
fn = r"C:\Users\yahia\Downloads\server.log.2023-01-22"
out_lst = []
with  open (fn, 'rt', encoding='utf8') as f:
    line_no = 1

    while True:
        txt = f.readline()
        if not txt:
            break
        # line_no += 1
        if line_no > 2000: break
        # if line_no % 1000000 == 0: print (line_no)
        if txt[24:29] != 'ERROR': 
            continue
        x = txt.find(']', 32)
        # print (x, ':', txt)
        log_type = txt[30:x+1] # skip (default task-9999)
        log_desc = txt[x+22:].lstrip()
        if log_type == '[eg.intercom.hdb.rer.web.config.WebRequestInterceptor]': continue
        if log_type == '[org.thymeleaf.TemplateEngine]': 
            log_desc = log_desc[31:].lstrip()    # skip task-id in [THYMELEAF][default task-10430]
        x = log_desc.find('org.springframework.')
        if x != -1: # found
            y = log_desc[x:].find(':')
            log_desc_type = log_desc[x:x+y].split('.')[-1]
            print (y, log_desc[x:x+y+1], "***", log_desc_type)
        else:
            log_desc_type = ''
        out_lst.append([line_no, log_type, log_desc_type, log_desc])
        # print ([line_no, log_type, txt[x+2:]])
        out_lst.append([line_no, log_type, log_desc])
        
df = pd.DataFrame(out_lst, columns=['line_no', 'log_type', 'log_desc_type', 'log_desc'])
# x =df.pivot_table(columns=  'excption_type', index='excption_desc', values = 'line_no', aggfunc='count', margins=True, fill_value=0)
x =df.pivot_table(index= ['log_type', 'log_desc_type', 'log_desc'], values = 'line_no', aggfunc='count', margins=True, fill_value=0)
# df.to_csv('./out/xx1.csv', index=True)
x.to_csv('./out/xx.csv', index=True)
# x


# parse engine

In [ ]:
def test_parse_tech_log():
    fn = r"C:\Users\yahia\Downloads\server.log.2023-01-22"
    out_lst = []
    with  open (fn, 'rt', encoding='utf8') as f:
        line_no = 1

        while True:
            txt = f.readline()
            if not txt:
                break
            # line_no += 1
            if line_no > 2000: break
            # if line_no % 1000000 == 0: print (line_no)
            if txt[24:29] != 'ERROR': 
                continue

In [ ]:
Tokens = pd.DataFrame(columns = ['token','categ','prio','desc', 'func'], 
data=[
['ERROR [eg.intercom.hdb.rer.web.controllers.advice.ControllerExceptionHandler]', 'tech', 0, 'Exception', 'error_exception_handler'],
['ERROR [org.thymeleaf.TemplateEngine]', 'tech', 1, 'thymeleaf', 'thymeleaf_log_handler'],
['ERROR [io.undertow.request]', 'tech', 1, 'TemplateInputException', 'undertow_log_handler'],
['ERROR [eg.intercom.hdb.rer.web.config.WebRequestInterceptor]', 'user', 1, 'NID data', 'WebRequestInterceptor'],
['ERROR [eg.intercom.hdb.rer.web.config.security.CustomUsernamePasswordAuthenticationFilter]', 'user', 1, 'Authentication', 'authetication_error_handler'],
])
x = """
# ['ControllerExceptionHandler','tech',9,'ControllerExceptionHandler'],
['userLocked','user',0,None, None],
['MailSendException','user',1,'Email quota exception', None],
['UserNotFound','user',2,None], None,
['isReCAPTCHAValid','user',3,None, None],
['changeUserPassword','user',4,None, None],
['ResourceAccessException','user',4,'sending SMS', None],
['AuthenticationFailedException','user',4,'Too many login attempts', None],
['HttpRequestMethodNotSupportedException','tech',4,None, None],
['TemplateInputException','tech',5,None, None],
['IllegalStateException','tech',6,'ISE', None],
['[stderr], (default task-,tech',7,'stderr', None],
['Exception processing template','tech',8,'EPT', None],
['ControllerExceptionHandler','tech',9,'ControllerExceptionHandler', None],
['[stderr], (pool-','tech',10,'stderr', None],
['mimeType','tech',11,'java.lang', None],
['java.lang.','tech',12,'mimeType', None],
['java.base','tech',13,'java.base', None],
['SMSStatusUpdateJobScheduler','tech',14,'SMSStatusUpdateJobScheduler', None],
['.jboss.','tech',15,'jboss', None],
['org.wildfly.','tech', 16, 'Wildfly', None, None],
['org.springframework.', 'tech', 17, 'SpringFramework', None],
['io.undertow.', 'tech', 18, 'Undertow Web Server', None],
['javax.', 'tech', 19, 'javax', None],
['jdk.internal.reflect.GeneratedMethodAccessor', 'tech', 19, 'jdk', None],
['eg.intercom.hdb.rer.', 'tech', 19, 'Intercom', None],
['io.opentracing.contrib.jaxrs2.server.SpanFinishingFilter', 'tech', 19, 'jaxrs2', None],
['deployment.hdb-rer-admin', 'tech', 19, 'Admin CSV', None],
['java.net.UnknownHostException: www.google.com', 'tech', 19, 'UnknownHostException', None],
])
"""

In [ ]:
# from tokens_data import Tokens

def parse_tech_log_line(line_no, txt):
    for token in Tokens.itertuples():
        p = txt.find(token.token) 
        if p != -1 :  # found
            # print (token, "------->", token) 
            # error_token = token.token
            if token.func:
                l = len(token.token)
                match (token.func):
                    case 'error_exception_handler':
                        log_desc_type, log_desc = error_exception_handler(line_no, txt[p+l:])
                        return token.token, log_desc_type, log_desc
                    case 'thymeleaf_log_handler':
                        return token.token, token.desc, ''
                    case 'undertow_log_handler':
                        return token.token, token.desc, ''
                    case 'WebRequestInterceptor':
                        return token.token, token.desc, ''
                    case 'authetication_error_handler':
                        return token.token, token.desc, ''
                
            if pd.isnull(token.desc):
                error_token = token.token
            else:
                error_token = token.desc
            error_categ = token.categ

            # txt= None   # error text is not needed in this case
            classified = True
            break    
    return 'Unclassified', txt, None


In [ ]:
def error_exception_handler(line_no, txt):
    # found = 'ERROR [eg.intercom.hdb.rer.web.controllers.advice.ControllerExceptionHandler]'
    # Handling exception of type: org.springframework.web.HttpRequestMethodNotSupportedException: Request method 'GET' not supported
    lst = txt.split(': ')
    l = len(lst)
    try:
        if l ==1:   #(default task-7437) handling NotFoundException"
            exception_type = lst[0].split()[-1] 
            exception_desc = ''
        elif l == 2:
            # (default task-7604) Handling exception of type: java.lang.Exception
            exception_type = "Java Lang"     #lst[0]
            exception_desc = lst[1].rstrip()
        else:
            exception_type = lst[1].split('.')[-1]
            exception_desc = lst[2].rstrip()
    except Exception as e:
        print (line_no, ":", len(lst) , f", Exception: {e}\n", txt )

    # print (line_no, l,  exception_type, exception_desc, '\n', txt)
    return exception_type, exception_desc
    

def thymeleaf_log_handler(txt):
    return 0, 0
    # x=0
    # log_type = txt[30:x+1] # skip (default task-9999)
    # log_desc = txt[x+22:].lstrip()
    # if log_type == '[org.thymeleaf.TemplateEngine]': 
    #     log_desc = log_desc[31:].lstrip()    # skip task-id in [THYMELEAF][default task-10430]
    # x = log_desc.find('org.springframework.')
    # if x != -1: # found
    #     y = log_desc[x:].find(':')
    #     log_desc_type = log_desc[x:x+y].split('.')[-1]
    #     print (y, log_desc[x:x+y+1], "***", log_desc_type)
    # else:
    #     log_desc_type = ''
    # pass

In [ ]:

# fn = r"C:\Users\yahia\Downloads\18-06.txt"
fn = r"C:\Users\yahia\Downloads\server.log.2023-01-22"
out_lst = []
with  open (fn, 'rt', encoding='utf8') as f:
    line_no = 1

    while True:
        txt = f.readline()
        if not txt:
            break
        line_no += 1
        # if line_no > 2000000: break
        # if line_no % 1000000 == 0: print (line_no)
        if txt[24:29] != 'ERROR': 
            continue
        log_type, log_desc_type, log_desc=  parse_tech_log_line(line_no, txt)
        out_lst.append([line_no, log_type, log_desc_type, log_desc])
        
df = pd.DataFrame(out_lst, columns=['line_no', 'log_type', 'log_desc_type', 'log_desc'])
df.to_csv('./out/xx1.csv', index=False)
# x =df.pivot_table(columns=  'excption_type', index='excption_desc', values = 'line_no', aggfunc='count', margins=True, fill_value=0)
x =df.pivot_table(index= ['log_type', 'log_desc_type', 'log_desc'], values = 'line_no', aggfunc='count', margins=True, fill_value=0)
x.to_csv('./out/xx.csv', index=True)
# x

# test exception pivot

In [ ]:
fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-01-22.zip"
df = pd.read_csv(fn, compression='zip', low_memory=False)



In [ ]:
df.columns
df.error_line.fillna(' ', inplace = True)
df.loc[df.categ == 'tech'].pivot_table(index= ['service', 'error_line'], values = 'line_no', columns = 'token', aggfunc='count', fill_value=0)#.to_csv('./out/xxx.csv', index = True)

In [ ]:
df1 = df.loc[df.categ == 'tech']
df1.token.unique()
# df1.service.unique()
df1[['token', 'log_date']].groupby('token').aggregate('count')
# df1.loc[df1.token == 'Exception', ['service', 'log_date', 'error_line']].groupby(['service', 'error_line']).aggregate('count').sort_values('log_date', ascending=False)


# test pandas 2.0

In [ ]:
# fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-03.zip"
# fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\logs\server.log.2023-02-23.zip"
fn= r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-23-YJ-DELL.zip"
# %timeit df= pd.read_csv(fn)
df= pd.read_csv(fn, dtype_backend = 'pyarrow', low_memory=False)
# pd.options.mode.dtype_backend = 'pyarrow'


df.info()

In [ ]:
proj_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\Reservation with status N\Copy of mpc_pro.xls"
pro_2_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\Reservation with status N\Copy of soc_pro.xls"
data_types = {'الرقم القومى للزوج / ': 'string[pyarrow]', 
              'الرقم الأرضى': 'string[pyarrow]',
             'الرقم المحمول': 'string[pyarrow]',
             'الرقم القومى': 'string[pyarrow]',
             'رقم الأستمارة': 'string[pyarrow]'}
rdf_1= pd.read_excel(proj_fn, dtype_backend = 'pyarrow', skiprows=2 , dtype=data_types)
rdf_2= pd.read_excel(pro_2_fn, dtype_backend = 'pyarrow', skiprows=0 , dtype=data_types)
rdf =  pd.concat([rdf_1, rdf_2])
rdf.info()

In [ ]:
ip = df.query("token == 'confirmLandReservation True'")[['NID', 'IP_address', 'country', 'log_date']]
ip.info()

In [ ]:
df.query("token == 'confirmLandReservation True' and NID == '25006192300285'")

In [ ]:
rdf.merge(ip, how='left',right_on= 'NID', left_on=rdf.columns[15]).to_excel('./out/reservatio_with_N.xlsx')

In [5]:
lst=[1,2,3,4,5,6,7]
li = [100,122,233,222,333,444, 555]

pd.DataFrame(lst, index=li)

,0
100,1
122,2
233,3
222,4
333,5
444,6
555,7
